In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from sklearn.neural_network import MLPRegressor
import pickle as pickle

In [ ]:
with open("bsc-3m/traj_01_elab.pkl", "rb") as f:
    df = pickle.load(f)

In [ ]:
#id, class, xs, ys (or actually xs_640x360, ys_640x360 to plot it on the video/frame)
cols=["id","class","xs","ys", "xs_640x360", "ys_640x360", "times", "frame_in", "frame_out", "num_frames"]
data = df[cols]
data = data.loc[data['class'] == 'Car'] # select cars

In [ ]:
# traffic light data
with open('bsc-3m/signals.pickle', 'rb') as f:
    li = pickle.load(f)
    
with open('bsc-3m/signal_lines_true.pickle', 'rb') as f:
    asd = pickle.load(f)

In [ ]:
df = pd.read_csv('bsc-3m/signals_dense.csv')

### features
    x_coordinate x # normalize
    y_coordinate x
    x_direction  x more!
    y_direction  x
    x_dest       x
    y_dest       x
    light_color # how
    surrounding_traffic?
    time?

### target
    next_x       x as vector please
    next_y       x

In [ ]:
# position
x_pos = np.array([])
y_pos = np.array([])

# target position
x_tar = np.array([])
y_tar = np.array([])

# direction vectors
x_vec = np.array([])
y_vec = np.array([])

# destination

x_dest = np.array([])
y_dest = np.array([])

# times
times = np.array([])

# Frames step size
step_size = 15

for i in range(len(data)):
    # current row
    a = np.array(data['xs'].iloc[i][::step_size])
    b = np.array(data['ys'].iloc[i][::step_size])
    
    # compute position
    x_pos = np.concatenate((x_pos, a[:-1]))
    y_pos = np.concatenate((y_pos, b[:-1]))
    
    # compute target vector
    x_tar = np.concatenate((x_tar, a[1:]-a[:-1]))
    y_tar = np.concatenate((y_tar, b[1:]-b[:-1]))

    # compute direction vectors
    x_prev = np.concatenate(([a[0]], a[:-2]))
    y_prev = np.concatenate(([b[0]], b[:-2]))
    x_vec = np.concatenate((x_vec, (a[:-1] - x_prev)))
    y_vec = np.concatenate((y_vec, (b[:-1] - y_prev)))
    
    # destination
    x_dest = np.concatenate((x_dest, [a[-1]]*(len(a)-1)))
    y_dest = np.concatenate((y_dest, [b[-1]]*(len(b)-1)))
    
    # times
    times = np.concatenate((times, data['times'].iloc[i][::step_size][:-1]))

In [ ]:
df = pd.DataFrame({'x_pos' : x_pos, 'y_pos' : y_pos,
              'x_vec' : x_vec, 'y_vec' : y_vec,
              'x_tar' : x_tar, 'y_tar' : y_tar,
              'x_dest' : x_dest, 'y_dest' : y_dest,
              'times' : times})

with open('training.pkl', 'wb') as f:
    pickle.dump(df, f)